In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import config

In [ ]:
from enum import Enum
class DFUData(Enum):
    L = 0 # Left Foot
    R = 1 # Right Foot
    L_LCA = 2  # Angiosom LCA Left Foot 
    R_LCA = 3  # Angiosom LCA Right Foot
    L_LPA = 4  # Angiosom LPA Left Foot 
    R_LPA = 5  # Angiosom LPA Right Foot 
    L_MCA = 6  # Angiosom MCA Left Foot 
    R_MCA = 7  # Angiosom MCA Right Foot 
    L_MPA = 8  # Angiosom MPA Left Foot 
    R_MPA = 9  # Angiosom MPA Right Foot 


class INAOEDataset():
    def __init__(self, root_path, dm_group = False):
        cg_path = os.path.join(root_path, 'Control Group')
        dm_path = os.path.join(root_path, 'DM Group')

        self.dm_group = dm_group

        self.cg_subjects = sorted(list(map(lambda x: os.path.join(cg_path, x), os.listdir(cg_path))))
        self.dm_subjects = sorted(list(map(lambda x: os.path.join(dm_path, x), os.listdir(dm_path))))

    def set_group(self, dm_group):
        self.dm_group = dm_group

    def __len__(self):
        if self.dm_group:
            return len(self.dm_subjects)
        
        return len(self.cg_subjects) 

    def get_subjects_names(self):
        dataset = self.dm_subjects if self.dm_group else self.cg_subjects
        return list(map(lambda x: os.path.basename(x), dataset))

    def get_subject(self, idx, data:DFUData):
        dataset = self.dm_subjects if self.dm_group else self.cg_subjects

        subject = os.path.basename(dataset[idx])
        path = os.path.join(dataset[idx], 'Angiosoms') if data.value > 1 else dataset[idx]
        return pd.read_csv(os.path.join(path, subject + '_' + data.name + '.csv'), header=None).astype('float32')

In [ ]:
dataset = INAOEDataset(config.INAOE_DATASET_DIR)

In [ ]:
from scipy.stats import skew, kurtosis

def get_metrics(dataset, data:DFUData):
    subjects = dataset.get_subjects_names()
    mean_data = []
    std_data = []
    max_data = []
    min_data = []
    skew_data = []
    kurtosis_data = []
    metrics = []

    fulldata = None
    for idx in range(len(dataset)):
        # data = dataset.get_subject(idx, data).to_numpy()
        subject_data = dataset.get_subject(idx, data).to_numpy()
        subject_data = subject_data[subject_data>0.5] # Remove background
        mean_data.append(subject_data.mean())
        std_data.append(subject_data.std())
        max_data.append(subject_data.max())
        min_data.append(subject_data.min())
        skew_data.append(skew(subject_data))
        kurtosis_data.append(kurtosis(subject_data))

    metrics = {'Subject': subjects,
        '{}_mean'.format(data.name): mean_data, 
        '{}_std'.format(data.name): std_data,
        '{}_min'.format(data.name): min_data,
        '{}_max'.format(data.name): max_data,
        '{}_skew'.format(data.name): skew_data,
        '{}_kurtosis'.format(data.name): kurtosis_data}

    return pd.DataFrame(metrics)

## Qualitative Test

In [ ]:
plt.figure(figsize=(16, 8))
for i in range(len(dataset)):
    plt.subplot(5, 15, i+1)
    data = dataset.get_subject(i, DFUData.L_LCA)
    plt.imshow(data.to_numpy())
    plt.xticks([])
    plt.yticks([])

plt.show()

# MPA

In [ ]:
l_mpa_metrics = get_metrics(dataset, DFUData.L_MPA)
r_mpa_metrics = get_metrics(dataset, DFUData.R_MPA)

l_mpa_metrics.to_csv('data/MPA/L_MPA_metrics.csv', index=False)
r_mpa_metrics.to_csv('data/MPA/R_MPA_metrics.csv', index=False)

# LCA

In [ ]:
l_lca_metrics = get_metrics(dataset, DFUData.L_LCA)
r_lca_metrics = get_metrics(dataset, DFUData.R_LCA)

l_lca_metrics.to_csv('data/LCA/L_LCA_metrics.csv', index=False)
r_lca_metrics.to_csv('data/LCA/R_LCA_metrics.csv', index=False)

# LPA

In [ ]:
l_lpa_metrics = get_metrics(dataset, DFUData.L_LPA)
r_lpa_metrics = get_metrics(dataset, DFUData.R_LPA)

l_lpa_metrics.to_csv('data/LPA/L_LPA_metrics.csv', index=False)
r_lpa_metrics.to_csv('data/LPA/R_LPA_metrics.csv', index=False)

# MCA

In [ ]:
l_mca_metrics = get_metrics(dataset, DFUData.L_MCA)
r_mca_metrics = get_metrics(dataset, DFUData.R_MCA)

l_mca_metrics.to_csv('data/MCA/L_MCA_metrics.csv', index=False)
r_mca_metrics.to_csv('data/MCA/R_MCA_metrics.csv', index=False)

# Merge

# Full Foot

In [ ]:
l_metrics = get_metrics(dataset, DFUData.L)
r_metrics = get_metrics(dataset, DFUData.R)

l_metrics.to_csv('data/Full/L_metrics.csv', index=False)
r_metrics.to_csv('data/Full/R_metrics.csv', index=False)

# Angiosomes Metrics

In [ ]:
def angiosomes_metrics(dataset):
    angiosomes = [[DFUData.L_LCA, DFUData.R_LCA], [DFUData.L_LPA, DFUData.R_LPA], [DFUData.L_MCA, DFUData.R_MCA], [DFUData.L_MPA, DFUData.R_MPA]]
    
    angiosomes_names = ['LCA', 'LPA', 'MCA', 'MPA']
    subjects = dataset.get_subjects_names()

    for ang_idx, (left, right) in enumerate(angiosomes):
        mean_data = []
        std_data = []
        max_data = []
        min_data = []
        skew_data = []
        kurtosis_data = []
        current_angiosom = angiosomes_names[ang_idx]
        for idx in range(len(dataset)):
            l_data = dataset.get_subject(idx, left).to_numpy()
            r_data = dataset.get_subject(idx, right).to_numpy() 

            l_data = l_data[l_data>0.5] # Remove background
            r_data = r_data[r_data>0.5] # Remove background
            data = np.concatenate((l_data, r_data))

            mean_data.append(data.mean())
            std_data.append(data.std())
            max_data.append(data.max())
            min_data.append(data.min())
            skew_data.append(skew(data))
            kurtosis_data.append(kurtosis(data))

        metrics = {'Subjects': subjects,
            'mean': mean_data, 
            'std': std_data,
            'min': min_data,
            'max': max_data,
            'skew': skew_data,
            'kurtosis': kurtosis_data}

        df = pd.DataFrame(metrics)
        print('data/{}/{}_metrics.csv'.format(current_angiosom, current_angiosom))
        df.to_csv('data/{}/General_{}_metrics.csv'.format(current_angiosom, current_angiosom), index=False)

In [ ]:
angiosomes_metrics(dataset)

# General

In [ ]:
angiosomes = ['LCA', 'LPA', 'MCA', 'MPA']
mean_values = []
std_values = []
max_values = []
min_values = []

for angiosome in angiosomes:
    df = pd.read_csv('data/{}/General_{}_metrics.csv'.format(angiosome, angiosome))
    data = df['mean'].to_numpy()
    mean_values.append(data.mean())
    std_values.append(data.std())
    max_values.append(data.max())
    min_values.append(data.min())
    
metrics = {'angiosome': angiosomes,
        'mean': mean_values, 
        'std': std_values,
        'min': min_values,
        'max': max_values
    }

df = pd.DataFrame(metrics)
df.to_csv('data/General_metrics.csv', index=False)

In [ ]:
df = pd.read_csv('data/LCA/L_LCA_metrics.csv') 
df.iloc[:, 1].mean()

In [ ]:
df = pd.read_csv('data/LCA/R_LCA_metrics.csv') 
df.iloc[:, 1].mean()

In [ ]:
df1 = pd.read_csv('data/MPA/L_MPA_metrics.csv') 
df1.iloc[:, 1].mean()

In [ ]:
df2 = pd.read_csv('data/MPA/R_MPA_metrics.csv') 
df2.iloc[:, 1].mean()

In [ ]:
pd.merge(df1, df2, on='Subject')

In [ ]:
df1